### Indexing of Documents used for RAG

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
import os


# set parameters
#---------------------------------------------------------------------------------------
# general parameters
rag_docs_folder = "Rag_docs" # folder containing pdf files to be loaded

# test splitter parameters
chunk_size = 1000 # chunk size for splitting documents
chunk_overlap = 200 # chunk overlap for splitting documents
add_start_index = True # whether to track index in original document

# Qdrant parameters
min_len = 20 # minimum length of documents to be stored in Qdrant
collection_name = "demo_collection" # name of the Qdrant collection
db_path = "test_rag_db" # path to the Qdrant database

# embedding model parameters
model_name = "sentence-transformers/all-mpnet-base-v2" # embedding model name
#---------------------------------------------------------------------------------------


# get pdf files from Rag_docs folder 
paths = [path for path in os.listdir(rag_docs_folder) if path.endswith(".pdf")]

# list of docs, metadata and ids to store in Qdrant
docs = []
ids = []

# text splitter to split documents into smaller chunks for retrieval
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,  # chunk size (characters)
    chunk_overlap=chunk_overlap,  # chunk overlap (characters)
    add_start_index=add_start_index,  # track index in original document
    )

# load and process pdf files using PyPDFLoader
for path in paths:
    loader = PyPDFLoader(os.path.join(rag_docs_folder, path))
    documents = loader.load()

    # split documents into smaller chunks
    document_chunks = text_splitter.split_documents(documents)

    # only add documents to list that are longer than min_len characters
    documents_min_length = [d for d in document_chunks if len(d.page_content) > min_len]

    # add self-defined metadata to documents
    for doc in documents_min_length:
        doc.metadata = {"source": path}  

    # add documents and metadata to lists for Qdrant upload
    docs.extend(documents_min_length)

    print(f"Loaded {len(documents_min_length)} documents from {path}")

ids = [i for i in range(len(docs))]

# embedding model to convert text to vectors
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Initialize the Qdrant client with a local path
client = QdrantClient(path=db_path)

try:
    if not client.collection_exists(collection_name):
        # create a new collection if it doesn't exist

        vector_length = len(embeddings.embed_documents(["dummy"])[0])  # get vector length from dummy embedding
        client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=vector_length, distance=Distance.COSINE),
        )

    vector_store = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=embeddings,
    )

    _ = vector_store.add_documents(docs, ids=ids) # metadata is contained in the documents
finally:
    client.close()

Loaded 1424 documents from Beiersdorf-Annual-Report-2024.pdf
Loaded 62 documents from LOREAL_2024_Annual_Report.pdf
